In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

In [40]:
# Let's explore the data we have here

train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
stores = pd.read_csv("./stores.csv")
transactions = pd.read_csv("./transactions.csv")
h_days = pd.read_csv("./holidays_events.csv")
oil = pd.read_csv("./oil.csv")

In [41]:
train.head(5)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [42]:
def merge(df, m_df, on = 'date'):
 df = pd.merge(how = 'inner', on = on, left = df, right = m_df)

 return df

train = merge(train, oil, "date")
train = merge(train, h_days)
train = merge(train, stores, "store_nbr")
train = merge(train, transactions, ["store_nbr", "date"])

test = merge(test, oil, "date")
test = merge(test, h_days)
test = merge(test, stores, "store_nbr")
test = merge(test, transactions, ["store_nbr", "date"])

train = train.drop(['store_nbr'], axis = 1)
test = test.drop(['store_nbr'], axis = 1)

In [43]:

'2012-03-02' in h_days['date'].unique()

True

In [44]:
check = pd.to_datetime(['2018-10-26 12:00 -0530', '2018-10-26 12:00 -0500'],
               utc=True)

def split_date(df):
 all_hdays = h_days['date'].unique()

 df['is_holiday'] = df['date'].apply(
     lambda x: 1 if x in all_hdays else 0)
 
 df['date'] = pd.to_datetime(df['date'])

 df['day'] = df['date'].apply(lambda x: x.day)
 df['month'] = df['date'].apply(lambda x: x.month)
 df['year'] = df['date'].apply(lambda x: x.year)
 df['quarter'] = df['date'].apply(lambda x: x.quarter)

 df = df.drop(['date'], axis = 1)

 return df

train = split_date(train)
test = split_date(test)

In [45]:
def encode(df):
 le = LabelEncoder()
 cols_to_encode = ["family", "state", "type_holiday", "type_store", "locale", "locale_name", "transferred", "city"]

 for c in cols_to_encode:
  df[c] = le.fit_transform(df[c])
 
 return df

train = encode(train)
test = encode(test)

In [53]:
train["is_holiday"].unique()

array([1], dtype=int64)

In [47]:
train = train.drop(["type_store"], axis = 1)
test = test.drop(["type_store"], axis = 1)

In [48]:
# Fill the missing data:

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 322047 entries, 0 to 322046
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            322047 non-null  int64  
 1   family        322047 non-null  int32  
 2   sales         322047 non-null  float64
 3   onpromotion   322047 non-null  int64  
 4   dcoilwtico    300003 non-null  float64
 5   type_holiday  322047 non-null  int32  
 6   locale        322047 non-null  int32  
 7   locale_name   322047 non-null  int32  
 8   description   322047 non-null  object 
 9   transferred   322047 non-null  int64  
 10  city          322047 non-null  int32  
 11  state         322047 non-null  int32  
 12  cluster       322047 non-null  int64  
 13  transactions  322047 non-null  int64  
 14  is_holiday    322047 non-null  int64  
 15  day           322047 non-null  int64  
 16  month         322047 non-null  int64  
 17  year          322047 non-null  int64  
 18  quar

In [49]:
# Let's predict the missing values (for the oil column)

def predict_oil(df):

 df = df.drop(["description"], axis = 1)

 oil_test_data = df[df["dcoilwtico"].isnull()]
 copy = df.copy()
 copy.dropna(inplace=True)
 
 oil_y_train = copy["dcoilwtico"]
 oil_x_train = copy.drop(['dcoilwtico'], axis = 1)
 oil_x_test = oil_test_data.drop(['dcoilwtico'], axis = 1)

 model = LinearRegression()
 model.fit(oil_x_train, oil_y_train)

 oil_y_test = model.predict(oil_x_test)

 oil_y_test = np.round(oil_y_test, decimals = 0)

 print(oil_y_test)

 series_y_test = pd.Series(oil_y_test)

 indices = df[df["dcoilwtico"].isnull()].index

 for fill_index, dataframe_index in enumerate(indices):
  df.loc[dataframe_index, "dcoilwtico"] = oil_y_test[fill_index]

 return df

train = predict_oil(train)

[87. 87. 87. ... 34. 30. 34.]


In [50]:
train.isnull().mean() * 100

id              0.0
family          0.0
sales           0.0
onpromotion     0.0
dcoilwtico      0.0
type_holiday    0.0
locale          0.0
locale_name     0.0
transferred     0.0
city            0.0
state           0.0
cluster         0.0
transactions    0.0
is_holiday      0.0
day             0.0
month           0.0
year            0.0
quarter         0.0
dtype: float64

In [51]:
# Feature engineering

def f_eng(df):
 oil_mean_price = df['dcoilwtico'].describe()["mean"]
 df['oil_is_expensive'] = df["dcoilwtico"].apply(lambda x: 1 if x > oil_mean_price else 0 )
 df['after_paycheck'] = df['day'].apply(lambda x: 1 if (15 <= x <= 18) or (29 <= x <= 31) or (0 <= x <= 2) else 0)

 return df

train = f_eng(train)
test = f_eng(test)

In [52]:
train.corr().style.background_gradient("YlOrBr")


C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\pandas\io\formats\style.py:3555: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\pandas\io\formats\style.py:3556: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,id,family,sales,onpromotion,dcoilwtico,type_holiday,locale,locale_name,transferred,city,state,cluster,transactions,is_holiday,day,month,year,quarter,oil_is_expensive,after_paycheck
id,1.000000,0.000012,0.061237,0.181707,-0.845955,-0.009605,-0.047326,-0.038677,0.133468,0.002845,-0.004071,-0.018880,-0.018267,nan,0.050069,-0.051022,0.975936,-0.049458,-0.831285,-0.056285
family,0.000012,1.000000,-0.106843,-0.043938,-0.000089,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,nan,-0.000000,-0.000000,0.000000,-0.000000,0.000027,-0.000000
sales,0.061237,-0.106843,1.000000,0.418029,-0.063829,-0.034418,0.015988,-0.012931,-0.007167,0.051921,0.069463,0.033868,0.203691,nan,-0.000215,0.024787,0.053746,0.022671,-0.060156,0.008450
onpromotion,0.181707,-0.043938,0.418029,1.000000,-0.137518,-0.010783,0.003223,-0.014531,0.028325,0.009065,0.021061,0.005895,0.038163,nan,-0.001373,0.001239,0.175230,0.002912,-0.138081,-0.023330
dcoilwtico,-0.845955,-0.000089,-0.063829,-0.137518,1.000000,0.077308,-0.033028,0.097530,-0.044612,-0.002466,0.004080,0.021288,-0.019195,nan,-0.048948,-0.001902,-0.815699,0.003162,0.930978,-0.000987
type_holiday,-0.009605,0.000000,-0.034418,-0.010783,0.077308,1.000000,-0.163156,0.188937,0.110354,0.000202,0.001086,-0.000382,-0.211298,nan,-0.317886,-0.374644,0.077988,-0.323307,0.083043,0.013654
locale,-0.047326,0.000000,0.015988,0.003223,-0.033028,-0.163156,1.000000,-0.327203,0.040426,0.000529,-0.000083,-0.002219,0.058433,nan,-0.031217,-0.046330,-0.035274,-0.063097,0.001651,0.211014
locale_name,-0.038677,-0.000000,-0.012931,-0.014531,0.097530,0.188937,-0.327203,1.000000,-0.118137,-0.001338,-0.000117,0.002011,-0.033133,nan,-0.165468,0.169043,-0.070579,0.162456,0.088155,-0.094673
transferred,0.133468,0.000000,-0.007167,0.028325,-0.044612,0.110354,0.040426,-0.118137,1.000000,0.000494,-0.000503,-0.001880,-0.043902,nan,0.019258,-0.022338,0.133413,0.006871,-0.054695,-0.096792
city,0.002845,-0.000000,0.051921,0.009065,-0.002466,0.000202,0.000529,-0.001338,0.000494,1.000000,0.481413,0.085103,0.233240,nan,-0.000201,-0.003748,0.003519,-0.003659,-0.002633,0.001006
